In [11]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji
from sklearn.metrics import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]=="fake"):
            
            #if the predicited class is negative
            if(Y_pred[i]=="fake"):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]=="real"):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Actual Negative','Actual Positive'])
    
    Confusion_Matrix.rename(index={0: "Predicted Negative", 1: "Predicted Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)

In [13]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)


In [14]:
def preprocess_text(tweet, flag_stemm=True, flag_lemm=True, stop_words_list=None):
#     print(tweet)
    tweet=emoji.demojize(tweet)
    tweet=tweet.replace(":"," ")
#     print(tweet)
#     tweet=p.clean(tweet)
    ## clean (convert to lowercase and remove punctuations and characters and then strip and remove url)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
#     tweet = re.sub(r'19', 'Z', str(tweet).lower().strip())
#     tweet = re.sub(r'[0-9]+k*', ' X ', str(tweet).lower().strip())    
#     tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
#     print(tweet)

    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

In [15]:
df_training=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
df_training.drop(columns="id",inplace=True)
# df_training
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

In [16]:
df_validation=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))
df_validation.drop(columns="id",inplace=True)
# df_validation
df_validation['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_validation['tweet'])

In [7]:
df_test=pd.DataFrame(pd.read_csv("Constraint_English_Test - Sheet1.csv"))
# df_test
df_test['tweet_original']=df_test['tweet']
preprocess_text_caller(df_test['tweet'])

<ipython-input-3-fb9ae317fd00>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=2)

In [17]:
# Train_ngrams=vectorizer.fit_transform(df_training.tweet)
vectorizer.fit_transform(list(df_training.tweet) + list(df_validation.tweet) +list(df_test.tweet))
Train_ngrams=vectorizer.transform(df_training.tweet)

In [18]:
Validation_ngrams=vectorizer.transform(df_validation.tweet)

In [19]:
Validation_ngrams

<2140x28361 sparse matrix of type '<class 'numpy.float64'>'
	with 50472 stored elements in Compressed Sparse Row format>

In [20]:
Test_ngrams=vectorizer.transform(df_test.tweet)

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(Train_ngrams,df_training["label"])
y_pred_rf=rf.predict(Validation_ngrams)
f1_score(y_pred_rf,df_validation["label"],average="macro")

0.9214293769160768

In [22]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svc = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
svc.fit(Train_ngrams,df_training["label"])
y_pred_svm=svc.predict(Validation_ngrams)
f1_score(y_pred_svm,df_validation["label"],average="macro")

0.9420009074925841

In [23]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(random_state=0)
log.fit(Train_ngrams,df_training["label"])
y_pred_log=log.predict(Validation_ngrams)
f1_score(y_pred_log,df_validation["label"],average="macro")

0.9199220797238565

In [24]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300,verbose=True,activation="tanh",solver="lbfgs")
mlp.fit(Train_ngrams,df_training["label"])
y_pred_mlp=mlp.predict(Validation_ngrams)
f1_score(y_pred_mlp,df_validation["label"],average="macro")

0.93259630224574

In [25]:
from sklearn.ensemble import BaggingClassifier
svc1= make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
bag = BaggingClassifier(svc1,n_estimators=100,random_state=0,n_jobs=-1)
bag.fit(Train_ngrams,df_training["label"])
y_pred_bag=bag.predict(Validation_ngrams)
f1_score(y_pred_bag,df_validation["label"],average="macro")

0.9480886623094711

In [27]:
y=[]
for i in range(len(y_pred_svm)):
    if(y_pred_svm[i]==y_pred_bag[i]):
        y.append(y_pred_bag[i])
    elif(y_pred_bag[i]==y_pred_mlp[i]):
        y.append(y_pred_bag[i])
    elif(y_pred_svm[i]==y_pred_mlp[i]):
        y.append(y_pred_mlp[i])

In [28]:
f1_score(y,df_validation["label"],average="macro")

0.9480829440209819

In [ ]:
func_Eval(y,df_validation["label"])

In [104]:
func_Eval(y_pred_bag,df_validation["label"])

-----------------
Accuracy: 0.9481308411214954
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9330357142857143
-----------------
Recall: 0.9666975023126735
-----------------
F1 Score: 0.9495683780099955

For the Negative CLass, Label:0
-----------------
Precision: 0.9647058823529412
-----------------
Recall: 0.9291784702549575
-----------------
F1 Score: 0.9466089466089466

--------
Macro Average F1 0.9480886623094711
-----------------
Confusion Matrix:
                    Actual Negative  Actual Positive
Predicted Negative              984               75
Predicted Positive               36             1045


In [106]:
y_pred=bag.predict(Validation_ngrams)
print(f1_score(df_validation["label"],y_pred,average="macro"))
y_pred_test=bag.predict(Test_ngrams)
y_pred_test=pd.DataFrame(y_pred_test)
ans=pd.DataFrame(df_test["id"])
ans["label"]=y_pred_test
ans.to_csv("answer_bagging_9488.txt",index=False)

0.9480886623094711


In [30]:
import pickle
file=open("websites_test","rb")
websites_test=pickle.load(file)
file.close()

file=open("websites_train","rb")
websites_train=pickle.load(file)
file.close()

file=open("websites_valid","rb")
websites_valid=pickle.load(file)
file.close()

In [47]:
# len(websites)
from requests.exceptions import SSLError,ConnectionError
for i in tqdm(range(len(websites_train))):
    if("t.co" in websites_train[i][1]):
        try:
            websites_train[i][1]=requests.get(websites_train[i][1]).url
        except SSLError:
            print("error")
            continue
        except ConnectionError:
            continue
        except:
            continue

100%|██████████| 4314/4314 [00:00<00:00, 686661.89it/s]


In [34]:
import pickle
file=open("websites_train","wb")
pickle.dump(websites_train,file)
file.close()

In [38]:
# len(websites)
from requests.exceptions import SSLError
for i in tqdm(range(len(websites_valid))):
    if("t.co" in websites_valid[i][1]):
        try:
            websites_valid[i][1]=requests.get(websites_valid[i][1]).url
        except SSLError:
            print("error")
            continue
        except :
            continue

100%|██████████| 1459/1459 [00:00<00:00, 1116898.98it/s]


In [36]:
import pickle
file=open("websites_valid","wb")
pickle.dump(websites_valid,file)
file.close()

In [39]:
from requests.exceptions import SSLError
for i in tqdm(range(len(websites_test))):
    if("t.co" in websites_test[i][1]):

        try:
            websites_test[i][1]=requests.get(websites_test[i][1]).url
        except SSLError:
            print("error")
            continue
        except :
            continue

100%|██████████| 1464/1464 [00:00<00:00, 460269.92it/s]


In [40]:
import pickle
file=open("websites_test","wb")
pickle.dump(websites_test,file)
file.close()

In [87]:
web=set()
for i in websites_train:
    if("t.co" not in i[1]):
        web.add(i[1][:min(20,len(i[1]))].replace("http://","").replace("https://","").replace("www.",""))

In [88]:
len(web)

189

In [89]:

for i in websites_valid:
    if("t.co" not in i[1]):
        web.add(i[1][:min(20,len(i[1]))].replace("http://","").replace("https://","").replace("www.",""))

In [90]:
len(web2)

84

In [83]:
len(web)

694

In [91]:
for i in websites_test:
    if("t.co" not in i[1]):
        web.add(i[1][:min(20,len(i[1]))].replace("http://","").replace("https://","").replace("www.",""))

In [92]:
len(web)

240

In [93]:
web

{'1011now.',
 '11alive.',
 'aapf.org/aap',
 'abc.net.',
 'abc.us3.list',
 'accounts.goo',
 'acpjourn',
 'africacdc.or',
 'ageofaut',
 'aljazeer',
 'amp.cnn.com/',
 'amp.theatlan',
 'amp.wbur.org',
 'aninews.',
 'arogya.mahar',
 'assets.ctfas',
 'axios.co',
 'azdhs.go',
 'bbc.co.u',
 'bendigoa',
 'bethe1to',
 'betootaa',
 'blog.covidac',
 'blog.rootcla',
 'bloomber',
 'bmjopen.bmj.',
 'brighteo',
 'bu.edu/a',
 'business',
 'businessinth',
 'cbc.ca/n',
 'cbsnews.',
 'ccc19.org/',
 'cdc.gov/',
 'cddwesta',
 'cdph.ca.',
 'census.g',
 'civilavi',
 'cla.purd',
 'clinical',
 'clinowl.com/',
 'cms.gov/',
 'cnbc.com',
 'confirmsubsc',
 'consultqd.cl',
 'copcov.o',
 'coronavirus-',
 'coronavirus.',
 'covid.cdc.go',
 'covid19.govt',
 'covid19.ncdc',
 'covid19.tela',
 'covid19dashbo',
 'covid19respo',
 'covidactnow.',
 'covidly.com/',
 'covidtrackin',
 'crooked.com/',
 'customs.',
 'd4bl.org/',
 'd4bl.org/rep',
 'dailybusines',
 'dailyinterla',
 'dailyite',
 'dailymai',
 'docs.google.',
 'doh.wa.g

In [109]:
dic={}
j=0
for i in web:
    dic[i]=j
    j+=1

In [206]:
train_web=[]
tfidf_train_web=[]
for i in range(len(df_training)):
    temp=[]
    temp2=""
    for j in range(len(web)):
        temp.append(0)
    train_web.append(temp)
    tfidf_train_web.append(temp2)

In [207]:
count=0
for pair in websites_train:
    index=pair[0]
    website=pair[1]
    for i in web:
        if(i in website):
            count+=1
#             print(i)
            train_web[index][dic[i]]+=1
            tfidf_train_web[index]+=i+" "
#             print(train_web[index])
            break

In [208]:
docs=train_web
train_web=pd.DataFrame(train_web)


In [209]:
valid_web=[]
tfidf_valid_web=[]
for i in range(len(df_validation)):
    temp=[]
    temp2=""
    for j in range(len(web)):
        temp.append(0)
    valid_web.append(temp)
    tfidf_valid_web.append(temp2)

In [210]:
count=0
for pair in websites_valid:
    index=pair[0]
    website=pair[1]
    for i in web:
        if(i in website):
            count+=1
#             print(i)
            valid_web[index][dic[i]]+=1
            tfidf_valid_web[index]+=i+" "
#             print(valid_web[index])
            break

In [211]:
docs+=valid_web
valid_web=pd.DataFrame(valid_web)

In [219]:
test_web=[]
for i in range(len(df_test)):
    temp=[]
    temp2=""
    for j in range(len(web)):
        temp.append(0)
    test_web.append(temp)


In [220]:
count=0
for pair in websites_test:
    index=pair[0]
    website=pair[1]
    for i in web:
        if(i in website):
            count+=1
#             print(i)
            test_web[index][dic[i]]+=1
            break

In [221]:
docs+=test_web
test_web=pd.DataFrame(test_web)

In [222]:
# from scipy.sparse import csr_matrix, hstack

# csr_TrainFeature_without_ngram=csr_matrix(train_web)
# csr_ValidFeature_without_ngram=csr_matrix(valid_web)
# TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])
# ValidFeatureFinal=hstack([Validation_ngrams,csr_ValidFeature_without_ngram])

In [223]:
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

# clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
# clf.fit(TrainFeatureFinal,df_training["label"])
# y_pred=clf.predict(ValidFeatureFinal)

In [224]:
# f1_score(y_pred,df_validation["label"],average="macro")

In [225]:
# from sklearn.ensemble import BaggingClassifier
# svc1= make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
# bag = BaggingClassifier(svc1,n_estimators=100,random_state=0,n_jobs=-1)
# bag.fit(TrainFeatureFinal,df_training["label"])
# y_pred_bag=bag.predict(ValidFeatureFinal)
# f1_score(y_pred_bag,df_validation["label"],average="macro")

In [226]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tfidf.fit(docs)
train = tfidf.transform(train_web)
valid = tfidf.transform(valid_web)
test  = tfidf.transform(test_web) 

In [230]:
tfidf_TrainFeatureFinal=hstack([Train_ngrams,train])
tfidf_ValidFeatureFinal=hstack([Validation_ngrams,valid])
tfidf_TestFeatureFinal =hstack([Test_ngrams,test]) 

In [231]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
clf.fit(tfidf_TrainFeatureFinal,df_training["label"])
y_pred=clf.predict(tfidf_ValidFeatureFinal)

In [232]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9550805113614996

In [233]:
from sklearn.ensemble import BaggingClassifier
svc1= make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
bag = BaggingClassifier(svc1,n_estimators=100,random_state=0,n_jobs=-1)
bag.fit(tfidf_TrainFeatureFinal,df_training["label"])
y_pred_bag=bag.predict(tfidf_ValidFeatureFinal)
f1_score(y_pred_bag,df_validation["label"],average="macro")

0.9569606262307582

In [240]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(random_state=0)
log.fit(tfidf_TrainFeatureFinal,df_training["label"])
y_pred_log=log.predict(tfidf_ValidFeatureFinal)
f1_score(y_pred_log,df_validation["label"],average="macro")

0.9349130047299994

In [241]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300,verbose=True,activation="tanh",solver="lbfgs")
mlp.fit(Train_ngrams,df_training["label"])
y_pred_mlp=mlp.predict(Validation_ngrams)
f1_score(y_pred_mlp,df_validation["label"],average="macro")

0.93259630224574

In [243]:
y=[]
for i in range(len(y_pred_svm)):
    if(y_pred_svm[i]==y_pred_bag[i]):
        y.append(y_pred_bag[i])
    elif(y_pred_bag[i]==y_pred_mlp[i]):
        y.append(y_pred_bag[i])
    elif(y_pred_svm[i]==y_pred_mlp[i]):
        y.append(y_pred_mlp[i])
f1_score(y,df_validation["label"],average="macro")

0.9541537105501554

In [239]:
y_pred=bag.predict(tfidf_ValidFeatureFinal)
print(f1_score(df_validation["label"],y_pred,average="macro"))
y_pred_test=bag.predict(tfidf_TestFeatureFinal)
y_pred_test=pd.DataFrame(y_pred_test)
ans=pd.DataFrame(df_test["id"])
ans["label"]=y_pred_test
ans.to_csv("answer_9569_valid_web.txt",index=False)

0.9569606262307582
